# Pruebas API 

##### **En este notebook, vamos a realizar las pruebas necesaria y modificar nuestros archivos(DataFrames) para optimizar las funciones en su deploy con Render**

In [1]:
from fastapi import FastAPI
from typing import List, Dict
import pandas as pd
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')

app = FastAPI()

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\hp\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [2]:
# Cargamos los DataFrames que usaremos para nuestras funciones 
games = pd.read_parquet('C:/Users/hp/Desktop/HENRY/Soy henry Modulos/PI ML/PI MLOps - STEAM/datasets limpios/steam_games_limpio')
reviews = pd.read_parquet('C:/Users/hp/Desktop/HENRY/Soy henry Modulos/PI ML/PI MLOps - STEAM/datasets limpios/user_reviews_limpio')
items = pd.read_parquet('C:/Users/hp/Desktop/HENRY/Soy henry Modulos/PI ML/PI MLOps - STEAM/datasets limpios/users_items_limpio')

In [6]:
games['id'] = games['id'].astype(int)
items['item_id'] = items['item_id'].astype(int)
reviews['item_id'] = reviews['item_id'].astype(int)

* #### FUNCION 1

In [7]:
# Unir el DataFrame games con el DataFrame items en la columna id para que nuestra funcion no sea tan pesada y sea optima en nuestra api 
df_funcion_1= pd.merge(games, items, left_on='id', right_on='item_id')
df_funcion_1

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,id,user_id,items_count,steam_id,user_url,item_id,item_name,playtime_forever,playtime_2weeks
0,Stainless Games Ltd,"[Action, Indie, Racing]",Carmageddon Max Pack,Carmageddon Max Pack,http://store.steampowered.com/app/282010/Carma...,1997-06-30,"[Racing, Action, Classic, Indie, Gore, 1990's,...",http://steamcommunity.com/app/282010/reviews/?...,282010,UTNerd24,188,76561198053985682,http://steamcommunity.com/id/UTNerd24,282010,Carmageddon Max Pack,5.0,0.0
1,Stainless Games Ltd,"[Action, Indie, Racing]",Carmageddon Max Pack,Carmageddon Max Pack,http://store.steampowered.com/app/282010/Carma...,1997-06-30,"[Racing, Action, Classic, Indie, Gore, 1990's,...",http://steamcommunity.com/app/282010/reviews/?...,282010,I_DID_911_JUST_SAYING,154,76561198067520555,http://steamcommunity.com/id/I_DID_911_JUST_SA...,282010,Carmageddon Max Pack,0.0,0.0
2,Stainless Games Ltd,"[Action, Indie, Racing]",Carmageddon Max Pack,Carmageddon Max Pack,http://store.steampowered.com/app/282010/Carma...,1997-06-30,"[Racing, Action, Classic, Indie, Gore, 1990's,...",http://steamcommunity.com/app/282010/reviews/?...,282010,76561197962104795,79,76561197962104795,http://steamcommunity.com/profiles/76561197962...,282010,Carmageddon Max Pack,0.0,0.0
3,Stainless Games Ltd,"[Action, Indie, Racing]",Carmageddon Max Pack,Carmageddon Max Pack,http://store.steampowered.com/app/282010/Carma...,1997-06-30,"[Racing, Action, Classic, Indie, Gore, 1990's,...",http://steamcommunity.com/app/282010/reviews/?...,282010,r3ap3r78,331,76561198011684252,http://steamcommunity.com/id/r3ap3r78,282010,Carmageddon Max Pack,0.0,0.0
4,Stainless Games Ltd,"[Action, Indie, Racing]",Carmageddon Max Pack,Carmageddon Max Pack,http://store.steampowered.com/app/282010/Carma...,1997-06-30,"[Racing, Action, Classic, Indie, Gore, 1990's,...",http://steamcommunity.com/app/282010/reviews/?...,282010,saint556,583,76561198072532181,http://steamcommunity.com/id/saint556,282010,Carmageddon Max Pack,13.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4120770,Valve,[Action],Counter-Strike: Condition Zero,Counter-Strike: Condition Zero,http://store.steampowered.com/app/80/CounterSt...,2004-03-01,"[Action, FPS, Shooter, Multiplayer, Singleplay...",http://steamcommunity.com/app/80/reviews/?brow...,80,76561198273508956,15,76561198273508956,http://steamcommunity.com/profiles/76561198273...,80,Counter-Strike: Condition Zero,0.0,0.0
4120771,Valve,[Action],Counter-Strike: Condition Zero,Counter-Strike: Condition Zero,http://store.steampowered.com/app/80/CounterSt...,2004-03-01,"[Action, FPS, Shooter, Multiplayer, Singleplay...",http://steamcommunity.com/app/80/reviews/?brow...,80,76561198282090798,5,76561198282090798,http://steamcommunity.com/profiles/76561198282...,80,Counter-Strike: Condition Zero,0.0,0.0
4120772,Valve,[Action],Counter-Strike: Condition Zero,Counter-Strike: Condition Zero,http://store.steampowered.com/app/80/CounterSt...,2004-03-01,"[Action, FPS, Shooter, Multiplayer, Singleplay...",http://steamcommunity.com/app/80/reviews/?brow...,80,943525,58,76561198283103833,http://steamcommunity.com/id/943525,80,Counter-Strike: Condition Zero,0.0,0.0
4120773,Valve,[Action],Counter-Strike: Condition Zero,Counter-Strike: Condition Zero,http://store.steampowered.com/app/80/CounterSt...,2004-03-01,"[Action, FPS, Shooter, Multiplayer, Singleplay...",http://steamcommunity.com/app/80/reviews/?brow...,80,76561198283312749,49,76561198283312749,http://steamcommunity.com/profiles/76561198283...,80,Counter-Strike: Condition Zero,9.0,0.0


In [8]:
# eliminamos las columnas que nos serviran para nuestra primer funcion
df_funcion_1.drop(columns=['publisher','tags','url','reviews_url','items_count','steam_id','user_url','playtime_2weeks','item_name','user_id','title','app_name'], axis=1, inplace=True)

In [9]:
# extraemos solo el año de la columna release_date
df_funcion_1['año'] = pd.to_datetime(df_funcion_1['release_date']).dt.year


In [10]:
# eliminamos las columnas release_date e item_id 
df_funcion_1.drop(columns=['release_date','item_id'],axis=1,inplace=True)
df_funcion_1

,genres,id,playtime_forever,año
0,"[Action, Indie, Racing]",282010,5.0,1997
1,"[Action, Indie, Racing]",282010,0.0,1997
2,"[Action, Indie, Racing]",282010,0.0,1997
3,"[Action, Indie, Racing]",282010,0.0,1997
4,"[Action, Indie, Racing]",282010,13.0,1997
...,...,...,...,...
4120770,[Action],80,0.0,2004
4120771,[Action],80,0.0,2004
4120772,[Action],80,0.0,2004
4120773,[Action],80,9.0,2004


In [50]:
df_funcion_1 = df_funcion_1.explode('genres')
df_funcion_1.to_parquet('funcion1.parquet')

In [12]:
@app.get("/PlayTimeGenre/{genero}")
def PlayTimeGenre(genero: str) -> Dict[str, int]:
    # Filtrar los datos por el género especificado
    df_genero = df_funcion_1[df_funcion_1['genres'] == genero]
    
    # Agrupamos los datos por año
    df_grouped = df_genero.groupby('año').agg({'playtime_forever': 'sum'}).reset_index()

    # Encontrar el año con el tiempo de juego más alto
    mayor_tiempo_jugado = df_grouped[df_grouped['playtime_forever'] == df_grouped['playtime_forever'].max()]['año'].values[0]
    
    return {f"Año de lanzamiento con más horas jugadas para {genero}" : mayor_tiempo_jugado}


In [13]:
PlayTimeGenre('RPG')

{'Año de lanzamiento con más horas jugadas para RPG': 2011}

* #### FUNCION 2

In [14]:
df_funcion_2= pd.merge(games, items, left_on='id', right_on='item_id')
df_funcion_2

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,id,user_id,items_count,steam_id,user_url,item_id,item_name,playtime_forever,playtime_2weeks
0,Stainless Games Ltd,"[Action, Indie, Racing]",Carmageddon Max Pack,Carmageddon Max Pack,http://store.steampowered.com/app/282010/Carma...,1997-06-30,"[Racing, Action, Classic, Indie, Gore, 1990's,...",http://steamcommunity.com/app/282010/reviews/?...,282010,UTNerd24,188,76561198053985682,http://steamcommunity.com/id/UTNerd24,282010,Carmageddon Max Pack,5.0,0.0
1,Stainless Games Ltd,"[Action, Indie, Racing]",Carmageddon Max Pack,Carmageddon Max Pack,http://store.steampowered.com/app/282010/Carma...,1997-06-30,"[Racing, Action, Classic, Indie, Gore, 1990's,...",http://steamcommunity.com/app/282010/reviews/?...,282010,I_DID_911_JUST_SAYING,154,76561198067520555,http://steamcommunity.com/id/I_DID_911_JUST_SA...,282010,Carmageddon Max Pack,0.0,0.0
2,Stainless Games Ltd,"[Action, Indie, Racing]",Carmageddon Max Pack,Carmageddon Max Pack,http://store.steampowered.com/app/282010/Carma...,1997-06-30,"[Racing, Action, Classic, Indie, Gore, 1990's,...",http://steamcommunity.com/app/282010/reviews/?...,282010,76561197962104795,79,76561197962104795,http://steamcommunity.com/profiles/76561197962...,282010,Carmageddon Max Pack,0.0,0.0
3,Stainless Games Ltd,"[Action, Indie, Racing]",Carmageddon Max Pack,Carmageddon Max Pack,http://store.steampowered.com/app/282010/Carma...,1997-06-30,"[Racing, Action, Classic, Indie, Gore, 1990's,...",http://steamcommunity.com/app/282010/reviews/?...,282010,r3ap3r78,331,76561198011684252,http://steamcommunity.com/id/r3ap3r78,282010,Carmageddon Max Pack,0.0,0.0
4,Stainless Games Ltd,"[Action, Indie, Racing]",Carmageddon Max Pack,Carmageddon Max Pack,http://store.steampowered.com/app/282010/Carma...,1997-06-30,"[Racing, Action, Classic, Indie, Gore, 1990's,...",http://steamcommunity.com/app/282010/reviews/?...,282010,saint556,583,76561198072532181,http://steamcommunity.com/id/saint556,282010,Carmageddon Max Pack,13.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4120770,Valve,[Action],Counter-Strike: Condition Zero,Counter-Strike: Condition Zero,http://store.steampowered.com/app/80/CounterSt...,2004-03-01,"[Action, FPS, Shooter, Multiplayer, Singleplay...",http://steamcommunity.com/app/80/reviews/?brow...,80,76561198273508956,15,76561198273508956,http://steamcommunity.com/profiles/76561198273...,80,Counter-Strike: Condition Zero,0.0,0.0
4120771,Valve,[Action],Counter-Strike: Condition Zero,Counter-Strike: Condition Zero,http://store.steampowered.com/app/80/CounterSt...,2004-03-01,"[Action, FPS, Shooter, Multiplayer, Singleplay...",http://steamcommunity.com/app/80/reviews/?brow...,80,76561198282090798,5,76561198282090798,http://steamcommunity.com/profiles/76561198282...,80,Counter-Strike: Condition Zero,0.0,0.0
4120772,Valve,[Action],Counter-Strike: Condition Zero,Counter-Strike: Condition Zero,http://store.steampowered.com/app/80/CounterSt...,2004-03-01,"[Action, FPS, Shooter, Multiplayer, Singleplay...",http://steamcommunity.com/app/80/reviews/?brow...,80,943525,58,76561198283103833,http://steamcommunity.com/id/943525,80,Counter-Strike: Condition Zero,0.0,0.0
4120773,Valve,[Action],Counter-Strike: Condition Zero,Counter-Strike: Condition Zero,http://store.steampowered.com/app/80/CounterSt...,2004-03-01,"[Action, FPS, Shooter, Multiplayer, Singleplay...",http://steamcommunity.com/app/80/reviews/?brow...,80,76561198283312749,49,76561198283312749,http://steamcommunity.com/profiles/76561198283...,80,Counter-Strike: Condition Zero,9.0,0.0


In [15]:
df_funcion_2 = df_funcion_2.explode('genres')
df_funcion_2['año'] = pd.to_datetime(df_funcion_2['release_date']).dt.year
df_funcion_2.drop(columns=['publisher','tags','url','reviews_url','items_count','release_date','item_id','steam_id','user_url','playtime_2weeks','item_name','title','app_name'], axis=1,inplace=True)

In [51]:
df_funcion_2.to_parquet('funcion2.parquet')

In [17]:
@app.get("/UserForGenre/{genero}")
def UserForGenre(genero: str) -> Dict[str, List[Dict[str, int]]]:
        # Filtrar los datos por el género 
    df_genero = df_funcion_2[df_funcion_2['genres'] == genero]
    
    # dividimos la columna playtime_forever para convertir el tiempo a horas ya que se encuentra en minutos
    df_genero.loc[:, 'playtime_forever'] = (df_genero['playtime_forever'] / 60).round(2)

    # Agrupamos los datos por user_id y año
    df_grouped = df_genero.groupby(['user_id', 'año']).agg({'playtime_forever': 'sum'}).reset_index()

    # Encontrar el usuario con el tiempo de juego más alto
    highest_playtime_user = df_grouped[df_grouped['playtime_forever'] == df_grouped['playtime_forever'].max()]['user_id'].values[0]

    # Crear una lista de la acumulación de horas jugadas por año para ese usuario
    user_data = df_grouped[df_grouped['user_id'] == highest_playtime_user]
    user_data = user_data.rename(columns={'playtime_forever': 'Horas'})
    horas_jugadas = user_data[['año', 'Horas']].to_dict('records')

    return {f"Usuario con más horas jugadas para Género: {genero}": highest_playtime_user, "Horas jugadas": horas_jugadas}
    



In [18]:
UserForGenre('Simulation')

{'Usuario con más horas jugadas para Género: Simulation': 'clawbot44',
 'Horas jugadas': [{'año': 2006, 'Horas': 12052.7},
  {'año': 2007, 'Horas': 17.74},
  {'año': 2008, 'Horas': 10.0},
  {'año': 2010, 'Horas': 6.74},
  {'año': 2011, 'Horas': 4.46},
  {'año': 2013, 'Horas': 149.26},
  {'año': 2014, 'Horas': 124.02},
  {'año': 2015, 'Horas': 824.0600000000001},
  {'año': 2016, 'Horas': 13.2},
  {'año': 2017, 'Horas': 104.78}]}

In [19]:
# Crear un analizador de sentimientos VADER
sid = SentimentIntensityAnalyzer()

# Función para asignar valores de sentimiento
def assign_sentiment_score(text):
    sentiment_scores = sid.polarity_scores(text)
    compound_score = sentiment_scores['compound']
    if compound_score >= 0.05:
        return 2  # Positivo
    elif compound_score <= -0.05:
        return 0  # Negativo
    else:
        return 1  # Neutral

# Aplicar el análisis de sentimiento y crear la nueva columna 'sentiment_analysis'
reviews['sentiment_analysis'] = reviews['review'].apply(assign_sentiment_score)

* #### FUNCION 3

In [20]:
df_funcion_3_y_4 = pd.merge(reviews,games,left_on='item_id',right_on='id')
df_funcion_3_y_4

,user_id,user_url,posted,last_edited,item_id,recommend,review,sentiment_analysis,publisher,genres,app_name,title,url,release_date,tags,reviews_url,id
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,2011-11-05,,1250,True,Simple yet with great replayability. In my opi...,2,Tripwire Interactive,[Action],Killing Floor,Killing Floor,http://store.steampowered.com/app/1250/Killing...,2009-05-14,"[FPS, Zombies, Co-op, Survival, Action, Multip...",http://steamcommunity.com/app/1250/reviews/?br...,1250
1,death-hunter,http://steamcommunity.com/id/death-hunter,2015-03-30,,1250,True,"Amazing, Non-stop action of blowing stuff to b...",2,Tripwire Interactive,[Action],Killing Floor,Killing Floor,http://store.steampowered.com/app/1250/Killing...,2009-05-14,"[FPS, Zombies, Co-op, Survival, Action, Multip...",http://steamcommunity.com/app/1250/reviews/?br...,1250
2,DJKamBer,http://steamcommunity.com/id/DJKamBer,2013-07-12,,1250,True,"Compared to Left 4 Dead 2, this game REALLY gi...",0,Tripwire Interactive,[Action],Killing Floor,Killing Floor,http://store.steampowered.com/app/1250/Killing...,2009-05-14,"[FPS, Zombies, Co-op, Survival, Action, Multip...",http://steamcommunity.com/app/1250/reviews/?br...,1250
3,diego9031,http://steamcommunity.com/id/diego9031,2015-08-13,,1250,True,Jogo ♥♥♥♥.,1,Tripwire Interactive,[Action],Killing Floor,Killing Floor,http://store.steampowered.com/app/1250/Killing...,2009-05-14,"[FPS, Zombies, Co-op, Survival, Action, Multip...",http://steamcommunity.com/app/1250/reviews/?br...,1250
4,76561198081962345,http://steamcommunity.com/profiles/76561198081...,2014-04-05,,1250,True,cara nas imagens esse jogo da pouco de medo ma...,1,Tripwire Interactive,[Action],Killing Floor,Killing Floor,http://store.steampowered.com/app/1250/Killing...,2009-05-14,"[FPS, Zombies, Co-op, Survival, Action, Multip...",http://steamcommunity.com/app/1250/reviews/?br...,1250
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40877,wendyfiore,http://steamcommunity.com/id/wendyfiore,2015-06-11,,367780,True,The game is a good game. I might understand th...,2,Soloweb Studios,"[Action, Indie]",Aero's Quest,Aero's Quest,http://store.steampowered.com/app/367780/Aeros...,2015-06-08,"[Action, Indie, Side Scroller, Platformer, Puz...",http://steamcommunity.com/app/367780/reviews/?...,367780
40878,danebuchanan,http://steamcommunity.com/id/danebuchanan,2015-06-13,,367780,True,I can understand why Aero's Quest brings out s...,2,Soloweb Studios,"[Action, Indie]",Aero's Quest,Aero's Quest,http://store.steampowered.com/app/367780/Aeros...,2015-06-08,"[Action, Indie, Side Scroller, Platformer, Puz...",http://steamcommunity.com/app/367780/reviews/?...,367780
40879,danebuchanan,http://steamcommunity.com/id/danebuchanan,2015-06-13,,305920,False,i really didn't like it. i'm sorry. slow and b...,0,"ShaunJS, Digerati Distribution","[Adventure, Indie]",Another Perspective,Another Perspective,http://store.steampowered.com/app/305920/Anoth...,2014-08-08,"[Indie, Adventure, Platformer, Puzzle, 2D, Puz...",http://steamcommunity.com/app/305920/reviews/?...,305920
40880,76561198194200105,http://steamcommunity.com/profiles/76561198194...,2015-08-11,,306040,True,"Simple and fun, great art work.",2,Antanas Marcelionis,"[Casual, Indie, Simulation, Strategy]",The Howler,The Howler,http://store.steampowered.com/app/306040/The_H...,2016-04-21,"[Indie, Casual, Steampunk, Simulation, Strateg...",http://steamcommunity.com/app/306040/reviews/?...,306040


In [22]:
df_funcion_3_y_4['año'] = pd.to_datetime(df_funcion_3_y_4['release_date']).dt.year
df_funcion_3_y_4

,user_id,user_url,posted,last_edited,item_id,recommend,review,sentiment_analysis,publisher,genres,app_name,title,url,release_date,tags,reviews_url,id,año
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,2011-11-05,,1250,True,Simple yet with great replayability. In my opi...,2,Tripwire Interactive,[Action],Killing Floor,Killing Floor,http://store.steampowered.com/app/1250/Killing...,2009-05-14,"[FPS, Zombies, Co-op, Survival, Action, Multip...",http://steamcommunity.com/app/1250/reviews/?br...,1250,2009
1,death-hunter,http://steamcommunity.com/id/death-hunter,2015-03-30,,1250,True,"Amazing, Non-stop action of blowing stuff to b...",2,Tripwire Interactive,[Action],Killing Floor,Killing Floor,http://store.steampowered.com/app/1250/Killing...,2009-05-14,"[FPS, Zombies, Co-op, Survival, Action, Multip...",http://steamcommunity.com/app/1250/reviews/?br...,1250,2009
2,DJKamBer,http://steamcommunity.com/id/DJKamBer,2013-07-12,,1250,True,"Compared to Left 4 Dead 2, this game REALLY gi...",0,Tripwire Interactive,[Action],Killing Floor,Killing Floor,http://store.steampowered.com/app/1250/Killing...,2009-05-14,"[FPS, Zombies, Co-op, Survival, Action, Multip...",http://steamcommunity.com/app/1250/reviews/?br...,1250,2009
3,diego9031,http://steamcommunity.com/id/diego9031,2015-08-13,,1250,True,Jogo ♥♥♥♥.,1,Tripwire Interactive,[Action],Killing Floor,Killing Floor,http://store.steampowered.com/app/1250/Killing...,2009-05-14,"[FPS, Zombies, Co-op, Survival, Action, Multip...",http://steamcommunity.com/app/1250/reviews/?br...,1250,2009
4,76561198081962345,http://steamcommunity.com/profiles/76561198081...,2014-04-05,,1250,True,cara nas imagens esse jogo da pouco de medo ma...,1,Tripwire Interactive,[Action],Killing Floor,Killing Floor,http://store.steampowered.com/app/1250/Killing...,2009-05-14,"[FPS, Zombies, Co-op, Survival, Action, Multip...",http://steamcommunity.com/app/1250/reviews/?br...,1250,2009
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40877,wendyfiore,http://steamcommunity.com/id/wendyfiore,2015-06-11,,367780,True,The game is a good game. I might understand th...,2,Soloweb Studios,"[Action, Indie]",Aero's Quest,Aero's Quest,http://store.steampowered.com/app/367780/Aeros...,2015-06-08,"[Action, Indie, Side Scroller, Platformer, Puz...",http://steamcommunity.com/app/367780/reviews/?...,367780,2015
40878,danebuchanan,http://steamcommunity.com/id/danebuchanan,2015-06-13,,367780,True,I can understand why Aero's Quest brings out s...,2,Soloweb Studios,"[Action, Indie]",Aero's Quest,Aero's Quest,http://store.steampowered.com/app/367780/Aeros...,2015-06-08,"[Action, Indie, Side Scroller, Platformer, Puz...",http://steamcommunity.com/app/367780/reviews/?...,367780,2015
40879,danebuchanan,http://steamcommunity.com/id/danebuchanan,2015-06-13,,305920,False,i really didn't like it. i'm sorry. slow and b...,0,"ShaunJS, Digerati Distribution","[Adventure, Indie]",Another Perspective,Another Perspective,http://store.steampowered.com/app/305920/Anoth...,2014-08-08,"[Indie, Adventure, Platformer, Puzzle, 2D, Puz...",http://steamcommunity.com/app/305920/reviews/?...,305920,2014
40880,76561198194200105,http://steamcommunity.com/profiles/76561198194...,2015-08-11,,306040,True,"Simple and fun, great art work.",2,Antanas Marcelionis,"[Casual, Indie, Simulation, Strategy]",The Howler,The Howler,http://store.steampowered.com/app/306040/The_H...,2016-04-21,"[Indie, Casual, Steampunk, Simulation, Strateg...",http://steamcommunity.com/app/306040/reviews/?...,306040,2016


In [23]:
df_funcion_3_y_4.drop(columns=['posted','user_url','last_edited','review','item_id','publisher','genres','url','release_date','tags','reviews_url'],axis=1,inplace=True)


In [52]:
df_funcion_3_y_4.to_parquet('funcion3y4.parquet')

In [57]:
@app.get("/UsersRecommend/{año}")
def UsersRecommend(año: int) -> List[Dict[str, str]]:
    # Filtrar las reseñas que son recomendaciones y tienen un análisis de sentimiento positivo o neutral
    filtro = df_funcion_3_y_4[(df_funcion_3_y_4['recommend'] == True) & (df_funcion_3_y_4['sentiment_analysis'].isin([1, 2]))]

    # Filtrar los juegos que se lanzaron en el año dado
    filtro = filtro[filtro['año'] == año]

    # Contar las recomendaciones para cada juego
    cuenta_de_recomendaciones = filtro['app_name'].value_counts()

    # Devolver los tres juegos con más recomendaciones
    top_juegos = cuenta_de_recomendaciones.nlargest(3).index.tolist()

    return [{"Puesto 1": top_juegos[0]}, {"Puesto 2": top_juegos[1]}, {"Puesto 3": top_juegos[2]}]
    



In [58]:
UsersRecommend(2011)

[{'Puesto 1': 'Terraria'},
 {'Puesto 2': 'Portal 2'},
 {'Puesto 3': 'No More Room in Hell'}]

* #### FUNCION 4

In [55]:
@app.get("/UsersNotRecommend/{año}")
def UsersNotRecommend(año: int) -> List[Dict[str, str]]:
    # Filtrar las reseñas que son recomendaciones y tienen un análisis de sentimiento positivo o neutral
    filtrado = df_funcion_3_y_4[(df_funcion_3_y_4['recommend'] == False) & (df_funcion_3_y_4['sentiment_analysis'].isin([0]))]

    # Filtrar los juegos que se lanzaron en el año dado
    filtrado = filtrado[filtrado['año'] == año]

    # Contar las recomendaciones para cada juego
    cuenta_negativa = filtrado['app_name'].value_counts()

    # Devolver los tres juegos con más recomendaciones
    top_juegos_norecomendados = cuenta_negativa.nlargest(3).index.tolist()

    return [{"Puesto 1": top_juegos_norecomendados[0]}, {"Puesto 2": top_juegos_norecomendados[1]}, {"Puesto 3": top_juegos_norecomendados[2]}]



In [56]:
UsersNotRecommend(2012)

[{'Puesto 1': 'Counter-Strike: Global Offensive'},
 {'Puesto 2': 'Realm of the Mad God'},
 {'Puesto 3': 'Ace of Spades: Battle Builder'}]

* #### FUNCION 5

In [53]:
@app.get("/sentiment_analysis/{año}")
def sentiment_analysis(año: int) -> Dict[str, int]:
    #filtramos el dataframe por año 
    df_filtrado = df_funcion_3_y_4[df_funcion_3_y_4['año'] == año]

    # Contar la cantidad de registros de reseñas que se encuentren categorizados con un análisis de sentimiento
    contar_sentimientos = df_filtrado['sentiment_analysis'].value_counts()
        # Crear un nuevo diccionario con el formato deseado
    salida_deseada = {
        f"En el año {año} hubo comentarios": {
            "positivos": contar_sentimientos.get(2, 0),
            "neutrales": contar_sentimientos.get(1, 0),
            "negativos": contar_sentimientos.get(0, 0)
        }
    }

    return salida_deseada


In [54]:
sentiment_analysis(2010)

{'En el año 2010 hubo comentarios': {'positivos': 1144,
  'neutrales': 290,
  'negativos': 263}}